# Setup

### additional dependencies/configuration

In [1]:
!pip install geopandas --user -q

### modules

In [2]:
import pyproj
import geopyspark as gps
import geopandas as gpd
import fiona

from functools import partial
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, Polygon, shape
from shapely.ops import transform

In [3]:
from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [4]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '8G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

### create overlay context

In [5]:
hou = an.Overlay('houston')

In [6]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [7]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

In [8]:
bounding = [hou.study_area['geom'][0]]

In [9]:
hou.build_layerset('polygon', 'base', polygons=bounding, pixel_value=1)

In [10]:
M.add_layer(hou.layersets['base'].mappable_layer)

### storm surge

In [11]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif'
mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

In [12]:
# construct a layerset from a tiff
hou.build_layerset('tiff', 'storm_surge', tiff = maskedtiff)

In [13]:
hou.layersets['storm_surge'].tiled = hou.layersets['storm_surge'].tiled.convert_data_type(gps.CellType.UINT8, 0).repartition(256) 

In [14]:
hou.layersets['storm_surge'].construct_map_layer()

In [15]:
# storm surge layer to the map
M.add_layer(hou.layersets['storm_surge'].mappable_layer)

### sea level rise

In [16]:
# download and unzip sea level rise polygons
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip'
d = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/"
# data.zip_to_dir(u, d)

In [17]:
# gdb
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [18]:
# replace with unquoted filepath to gdb in order to see layers
!ogrinfo /home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/

Had to open data source read-only.
INFO: Open of `/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/'
      using driver `OpenFileGDB' successful.
1: TX_HGX_low_0ft (Multi Polygon)
2: TX_HGX_low_1ft (Multi Polygon)
3: TX_HGX_low_2ft (Multi Polygon)
4: TX_HGX_low_3ft (Multi Polygon)
5: TX_HGX_low_4ft (Multi Polygon)
6: TX_HGX_low_5ft (Multi Polygon)
7: TX_HGX_low_6ft (Multi Polygon)
8: TX_HGX_slr_0ft (Multi Polygon)
9: TX_HGX_slr_1ft (Multi Polygon)
10: TX_HGX_slr_2ft (Multi Polygon)
11: TX_HGX_slr_3ft (Multi Polygon)
12: TX_HGX_slr_4ft (Multi Polygon)
13: TX_HGX_slr_5ft (Multi Polygon)
14: TX_HGX_slr_6ft (Multi Polygon)


In [19]:
sea_level_1 = data.read_from_gdb(gdb, 'TX_HGX_slr_1ft', MultiPolygon)

In [20]:
hou.build_layerset('polygon', 'sea_level', polygons=sea_level_1, pixel_value=1)

In [21]:
M.add_layer(hou.layersets['sea_level'].mappable_layer)

### flood

In [22]:
# output_shp = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_storm_surge.shp'
input_shp = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp'

In [23]:
clipped_shp = mask.clip_vector_to_boundaries(input_shp, hou)

In [24]:
# import geopandas as gpd
ct = data.read_from_shp(clipped_shp)

In [25]:
hou.build_layerset('polygon', 'flood', polygons = ct, pixel_value = 1)

In [26]:
M.add_layer(hou.layersets['flood'].mappable_layer)